In [8]:
import numpy as np
import pandas as pd
import pickle

import bs4 as bs
import urllib.request
import json
from bs4 import BeautifulSoup
import requests
import re

import MySQLdb, pickle
from sqlalchemy import create_engine

In [4]:
df = pd.DataFrame(columns = ['year', 'title','company','subname','mile','photos','video','exterior_color','interior_color','transmission','drivetrain','star','review_no','vendor','price'])

for page in range(1,50):
    url = 'https://www.cars.com/for-sale/searchresults.action/?page='+str(page)+'&perPage=100&rd=99999&searchSource=PAGINATION&showMore=true&sort=relevance&stkTypId=28881&zc=31216'

    sauce = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(sauce, 'lxml')

    specificSoup = soup.find_all('div', class_='listing-row__details')

    

    for div in specificSoup:
        name_index = div.find('h2', {'class' :'cui-delta listing-row__title'}).text
        name = name_index.split("\n")[1]

    
        year_index = re.findall('[0-9]{4}',name)[0:1]
        year = year_index[0]

        title_index = name.split(" ")[29:34]
        title = " ".join(title_index)

        company = title.split(" ")[0]
        
        try:
            subname = title.split(" ")[1]
        except:
            subname = company

        mile_index = div.find('span', {'class' : 'listing-row__mileage'}).text
        mile = mile_index.split("\n")[0]
        regex = re.compile("\d+")
        miles = regex.findall(mile) 
        mile = ''.join(miles)

        vendor_index =div.find('div',{'class' : 'listing-row__dealer-name listing-row__dealer-name-mobile'}).text
        vendor_group = vendor_index.split(' ')[0:4]

        vendor = ''
        for i in vendor_group:
            if i == '':continue
            elif re.findall('\n', i): vendor += re.sub('\n', '', i)
            else: vendor += " "+ i +" "

        photos_index = div.find('div', {'class' : 'media-count shadowed'}).text
        photos = re.findall('[0-9]{1,3}',photos_index.split("\n")[1])[0]

        video_index = div.find('div', {'class' : 'media-count shadowed'}).text
        try: 
            video = re.findall('[0-9]{1,3}',photos_index.split("\n")[2])[0]
        except:
            video = 0

        exterior_color = div.find('ul', {'class' : 'listing-row__meta'}).text
        try:
            exterior_color = re.sub('\n', ' ',exterior_color).split(" ")[4]
        except:
            exterior_color = 'black'
        
        interior_color = div.find('ul', {'class' : 'listing-row__meta'})
        try:
            interior_color = list(interior_color)[3].text.split(" ")[3:5]
        except:
            interior_color = "black"
        interior_color = " ".join(interior_color)
        interior_color = re.sub('/' , ' ' ,interior_color)

        transmission = div.find('ul', {'class' : 'listing-row__meta'})
        try:
            transmission = list(transmission)[5].text.split(" ")[2]
        except:
            transmission = "6-speed"
            

        drivetrain = div.find('ul', {'class' : 'listing-row__meta'})
        try:
            drivetrain = list(drivetrain)[7].text.split(" ")[2:5]
        except: 
            drivetrain = 'fwd'
        drivetrain = " ".join(drivetrain).lower()
        if drivetrain == 'rear wheel drive':
            drivetrain = 'rwd'
        elif drivetrain == 'front wheel drive':
            drivetrain = 'fwd'
        elif drivetrain == 'all wheel drive':
            drivetrain = '4wd'



        star_index =div.find('div',{'class' : 'dealer-rating-stars'}).text
        star = star_index.split(" ")[36]
        regex = re.compile("\d")
        star = regex.findall(star)[0] 


        review_index =div.find('span',{'class' : 'listing-row__review-number'}).text
        review_no = re.sub('\n', '',review_index.split(" ")[1])

        price_index = div.find('span', {'class' : 'listing-row__price'}).text
        price = price_index.split("\n")[1]
        regex = re.compile("\d")
        price = ''.join(regex.findall(price))
        price



        data = { 
                'year' : year,
                'title' : title.lower(),
                'company': company.lower(),
                'subname': subname.lower(),
                'mile' : mile,
                'photos': photos,
                'video' : video,
                'exterior_color' : exterior_color.lower(),
                'interior_color' : interior_color.lower(),
                'transmission' : transmission.lower(),
                'drivetrain' : drivetrain.lower(),
                'star': star,
                'review_no' : review_no,

                'vendor' : vendor.lower(),
                'price': price,
                    }


        df.loc[len(df)] = data
        
df['transmission'] = df['transmission'].apply(lambda x: '6-speed' if x == 'automatic' or x == '6' else x)
df['transmission'] = df['transmission'].apply(lambda x: '8-speed' if x == '8' else x)
df['transmission'] = df['transmission'].apply(lambda x: '5-speed' if x == '5' else x)   
df['transmission'] = df['transmission'].apply(lambda x: '5-speed' if x == '5' else x)   
df['transmission'] = df['transmission'].apply(lambda x: 'x-speed' if x != '1-speed' and x != '2-speed' and x != '3-speed' and \
                                              x != '4-speed' and x != '5-speed' and x != '6-speed' and x != '7-speed' and x != '7-speed' and\
                                              x != '8-speed' and x != '9-speed' and x != '10-speed' \
                                              else x)
df['drivetrain'] = df['drivetrain'].apply(lambda x: '4wd' if x == 'four wheel drive' or x == '4wd' or x=='4x4'or x=='awd'else x)
df['drivetrain'] = df['drivetrain'].apply(lambda x: 'fwd' if x == '2wd' or x=='f w d' else x)
df['drivetrain'] = df['drivetrain'].apply(lambda x: '4wd' if x != 'fwd' and  x!='rwd' and x!='4wd' else x)
df

,year,title,company,subname,mile,photos,video,exterior_color,interior_color,transmission,drivetrain,star,review_no,vendor,price
0,2012,mercedes-benz c 250 luxury,mercedes-benz,c,65374,32,1,black,cappuccino,7-speed,rwd,5,10,southern auto brokers,12998
1,2005,porsche 911 carrera s cabriolet,porsche,911,64034,32,1,silver,stone gray,6-speed,rwd,5,11,united imports inc,30990
2,2010,mercedes-benz e 550,mercedes-benz,e,58204,32,1,iridium,black,7-speed,rwd,5,26,inetwork auto group,18549
3,2007,chevrolet avalanche lt w/3lt,chevrolet,avalanche,41631,27,1,summit,ebony light cashmere,4-speed,4wd,5,21,minton motor cars,17999
4,2016,volkswagen jetta 1.4t s,volkswagen,jetta,46042,25,1,cardinal,black,6-speed,fwd,5,18,hertz car sales winston-salem,9998
5,2017,kia sportage lx,kia,sportage,43989,26,1,sparkling,black,6-speed,fwd,5,18,hertz car sales winston-salem,14998
6,2017,chevrolet malibu 1lt,chevrolet,malibu,47450,25,1,cajun,jet black,6-speed,fwd,5,42,hertz car sales orlando,13988
7,2016,nissan sentra s,nissan,sentra,48058,25,1,gray,gray,6-speed,fwd,5,42,hertz car sales orlando,9988
8,2008,volvo v70 3.2,volvo,v70,169982,32,1,burgundy,tan,6-speed,fwd,5,16,sharpmotors,4750
9,2009,pontiac g8 gt,pontiac,g8,88919,27,1,silver,black,6-speed,rwd,5,8,ez choice auto sales,14999


In [5]:
df["year"] = df["year"].astype('int')
df["mile"] = df["mile"].astype('int')
df["photos"] = df["photos"].astype('int')
df["video"] = df["video"].astype('int')
df["star"] = df["star"].astype('int')
df["review_no"] = df["review_no"].astype('int')
df["price"] = df["price"].astype('int')

In [11]:
class A():
    def __init__(self, data):
        self.data = data
    def disp(self):
        print(self.data)

In [12]:
pw = pickle.load(open('./Data/pw.p','rb'))

# write local car_info
engine = create_engine("mysql+mysqldb://root:" + pw.data + "@127.0.0.1/car_info")
df.to_sql(name="car_info", con=engine, if_exists='replace')